In [ ]:
import os

import geopandas as gpd
import pandas as pd
from odc.geo import BoundingBox

from coastlines.grids import INDONESIA_25, INDONESIA_CRS

os.environ["USE_PYGEOS"] = "0"

In [ ]:
# Select relevant tiles and convert to Geopandas
bounds = BoundingBox(94, -11, 142, 6, crs="EPSG:4326").to_crs(INDONESIA_CRS)
geom = INDONESIA_25.geojson(bbox=bounds)
tiles_projected = gpd.GeoDataFrame.from_features(geom, crs="EPSG:4326").to_crs(
    INDONESIA_CRS
)

tiles_projected.explore()

In [ ]:
# Get Indonesion boundaries from ucdavis
# Indo admin0 
url = "https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_IDN_0.json"
indo_adm = gpd.read_file(url)

# Clip admin0 to sample sites
sample_sites = "https://s3.ap-southeast-2.amazonaws.com/files.auspatious.com/coastlines/indonesia_four_sites.geojson"
indo_adm_sample = gpd.read_file(sample_sites)

indo_adm = indo_adm.clip(indo_adm_sample)
indo_adm_explode = indo_adm.explode()

indo_adm_explode.explore()

In [ ]:
# Prepare a dataset that contains continental coastlines with a 15 km buffer
area_limit = 1e9
indo = indo_adm_explode.to_crs(INDONESIA_CRS)
small_islands = indo[indo.geometry.area < area_limit]
large_islands = indo[indo.geometry.area >= area_limit]

# Buffer the long coastlines boundaries
coast = large_islands.boundary.buffer(15_000)
# And the small islands
islands = small_islands.buffer(15_000)

# Now merge them together
buffered = pd.concat([coast, islands])

aoi = gpd.GeoDataFrame(buffered, geometry=buffered.geometry).dissolve()
aoi.explore()

In [ ]:

# Clip tiles to AOI
coastal_tiles = tiles_projected.clip(aoi)
coastal_tiles.explore()


In [ ]:
# Dump a simple list of all tile_ids in a text file
coastal_tiles["idx"].to_csv(
    "../data/raw/indonesia_sample_tile_ids.txt", index=False, header=False
)

In [ ]:
# These tiles can be removed safely
SKIP_TILES = [
    # Edges and tiny areas
    "24,9",
    "24,11",
    "29,12",
    "30,12",
    "30,70",
    "45,5",
    "46,5",
    "58,9",
    "58,11",
    "59,8",
    "60,8",
    "63,0",
    "63,1",
    "65,3",
    "67,1",
    "67,2",
    # inland...
    "24,14",
    "24,13",
    "25,14",
    "25,13",
    "27,10",
    "27,14",
    "30,9",
    "30,10",
    "67,3",
]


coastal_tiles_filtered = coastal_tiles[~coastal_tiles["idx"].isin(SKIP_TILES)]
coastal_tiles_filtered.explore()

In [ ]:
# Export the results as GeoJSON
coastal_tiles_filtered.rename(columns={"idx": "id"})
coastal_tiles_filtered.to_crs("EPSG:4326").to_file(
    "../data/raw/indonesia_tiles.geojson", driver="GeoJSON"
)